In [1]:
import pandas as pd
import datetime

# Converting Daily Timeseries to a Weekly Timeseries

In [2]:
spy = pd.read_csv("outputs/SPY.csv")
spy = spy[["Date", "Close"]].rename(columns = {"Date": "day"})
spy["day"] = spy["day"].map(lambda x: pd.to_datetime(x))
spy.head()

,day,Close
0,2017-01-26,229.330002
1,2017-01-27,228.970001
2,2017-01-30,227.550003
3,2017-01-31,227.529999
4,2017-02-01,227.619995


## Creating match between days and weeks

In [3]:
min_spy_date, max_spy_date = spy.day.min(), spy.day.max()

matchDayWeek = pd.DataFrame()
matchDayWeek["day"] = pd.date_range(start = min_spy_date, end = max_spy_date)
matchDayWeek["weekNumber"] = matchDayWeek["day"].map(lambda x: int(x.strftime("%V")))
matchDayWeek["year"] = matchDayWeek["day"].map(lambda x: int(x.strftime("%Y")))

weekId = matchDayWeek[["year", "weekNumber"]].drop_duplicates()
weekId["weekId"] = range(weekId.shape[0])

matchDayWeek = matchDayWeek.merge(weekId, on = ["weekNumber", "year"]).drop(columns = ["weekNumber", "year"])

## Assigning delayed weekId on SPY to predict from YouTube Data

In [4]:
spyLabel = spy.merge(matchDayWeek, on = ["day"]).groupby("weekId").agg({"Close": ["first", "last"]}).reset_index()
spyLabel.columns = [c.replace("_", "") if c.startswith("_") else c for c in ["_".join(mc[::-1]) for mc in spyLabel.columns]]
spyLabel["ratioClose"] = 100 * (spyLabel["last_Close"] / spyLabel["first_Close"] - 1)
spyLabel["weekId"] = spyLabel["weekId"] - 1
spyLabel = spyLabel[["weekId", "ratioClose"]]
spyLabel.tail()

,weekId,ratioClose
280,281,-5.340105
281,282,-2.437337
282,283,4.001914
283,284,-1.891455
284,285,1.756734


In [5]:
spyLabel.to_csv("outputs/spy_label.csv", index = False)

# Numeric Cols FE for YouTube Data

In [6]:
videos_df = pd.read_csv("outputs/videoInfo.csv")
videos_df["shorts"] = videos_df["title"].map(lambda x: 1 if "#shorts" in x else 0)

In [7]:
numeric_selected_vars = [
    "channelTitle",
    "duration",
    "viewCount",
    "likeCount",
    "commentCount",
    "publishedAtDay",
    "publishedAtHour"
    ]

numeric_df = videos_df[numeric_selected_vars].copy()
numeric_df["percInteraction"] = numeric_df["commentCount"]/numeric_df["viewCount"]
numeric_df["percLike"] = numeric_df["likeCount"]/numeric_df["viewCount"]
numeric_df["ratioLikeComment"] = numeric_df["likeCount"]/numeric_df["commentCount"]

numeric_df.head()

,channelTitle,duration,viewCount,likeCount,commentCount,publishedAtDay,publishedAtHour,percInteraction,percLike,ratioLikeComment
0,Graham Stephan,896.0,25981.0,2109.0,389.0,2022-07-11,22,0.014972,0.081175,5.421594
1,Graham Stephan,727.0,440063.0,20145.0,1841.0,2022-07-08,22,0.004183,0.045778,10.942423
2,Graham Stephan,794.0,336090.0,14612.0,1498.0,2022-07-06,21,0.004457,0.043476,9.754339
3,Graham Stephan,775.0,677600.0,29372.0,1602.0,2022-07-01,20,0.002364,0.043347,18.334582
4,Graham Stephan,782.0,1133545.0,47011.0,2762.0,2022-06-29,21,0.002437,0.041473,17.020637


In [8]:
minCommonDate = numeric_df.groupby("channelTitle").agg({"publishedAtDay": "min"}).publishedAtDay.max()
maxCommonDate = numeric_df.groupby("channelTitle").agg({"publishedAtDay": "max"}).publishedAtDay.max()

raw_df = numeric_df[numeric_df.publishedAtDay >= minCommonDate].drop(columns = ["channelTitle"])
raw_df["day"] = raw_df["publishedAtDay"].map(lambda x: pd.to_datetime(x))

raw_df = raw_df.merge(matchDayWeek, on = ["day"]).drop(columns = ["publishedAtDay"])

In [9]:
numeric_cols_to_agg = [c for c in raw_df.columns if c not in ["day", "weekId"]]
dict_numeric_cols_to_agg = {c: ["max", "min", "mean", "sum", "std"] for c in numeric_cols_to_agg}

In [10]:
fe = raw_df.groupby("weekId").agg(dict_numeric_cols_to_agg).reset_index()
fe.columns = [c.replace("_", "") if c.startswith("_") else c for c in ["_".join(mc[::-1]) for mc in fe.columns]]

In [11]:
fe.head()

,weekId,max_duration,min_duration,mean_duration,sum_duration,std_duration,max_viewCount,min_viewCount,mean_viewCount,sum_viewCount,...,max_percLike,min_percLike,mean_percLike,sum_percLike,std_percLike,max_ratioLikeComment,min_ratioLikeComment,mean_ratioLikeComment,sum_ratioLikeComment,std_ratioLikeComment
0,101,996.0,959.0,977.500000,1955.0,26.162951,717847.0,165303.0,441575.000000,883150.0,...,0.096902,0.057767,0.077334,0.154669,0.027673,21.697130,6.594613,14.145872,28.291744,10.679092
1,102,862.0,618.0,753.666667,2261.0,124.275232,257457.0,72171.0,135933.666667,407801.0,...,0.061781,0.040589,0.051370,0.154109,0.010601,11.094406,2.765926,8.054048,24.162145,4.596763
2,103,1660.0,417.0,1035.500000,4142.0,579.811751,1093994.0,21314.0,403663.500000,1614654.0,...,0.069162,0.039082,0.049820,0.199282,0.013567,27.631325,5.387211,12.471994,49.887978,10.220347
3,104,979.0,279.0,706.250000,2825.0,300.094846,1331051.0,28061.0,372919.750000,1491679.0,...,0.077694,0.037418,0.058406,0.233623,0.019855,13.057990,2.952139,7.655605,30.622421,5.331848
4,105,1954.0,390.0,997.500000,3990.0,669.611081,366931.0,32877.0,208084.000000,832336.0,...,0.081119,0.034918,0.055470,0.221880,0.019098,18.708360,5.075536,10.515339,42.061354,6.125986


In [12]:
fe.to_csv("outputs/fe.csv", index = False)